This notebook creates masked unwrapped interferograms.

In [1]:
import os, glob
import shutil
import math
from osgeo import gdal
import h5py
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.path as pth

In [2]:
datadir = 'path/to/your/dir'
driver=gdal.GetDriverByName('ISCE')

# size of entire SLC
az0 = 0
daz= 3500
rg0 = 0
drg = 30000

# number of looks
alks= 7
rlks = 19

# dates of SLCs
dates = [os.path.basename(x) for x in glob.glob(f'{datadir}/SLC_vv/2*')]
dates = sorted(dates)
nd = len(dates)

In [3]:
# read in road array for masking
ds = gdal.Open(f'{datadir}/cropscape/arrays/Sentinel/roadarray.r4', gdal.GA_ReadOnly)
roadarray = ds.ReadAsArray()
roadarray[roadarray==0] = np.nan

In [4]:
# function for downlooking with NaNs
def boxcar_nan(image, size) -> np.ndarray:
    """Calculate the downsampled image with window "size", ignoring nans.

    Parameters
    ----------
    image : ndarray
        input image, 1 band, can be real or complex.
    size : int or tuple of int
        Window size. If a single int, the window is square.
        If a tuple of (row_size, col_size), the window can be rectangular.

    Returns
    -------
    ndarray
        image downlooked by size, simple average over the "size" window, ignoring nans.  
    """
    if isinstance(size, int):
        size = (size, size)
    if len(size) != 2:
        raise ValueError("size must be a single int or a tuple of 2 ints")
  
    row_size, col_size = size
 
    im_row_size, im_col_size = np.shape(image)
    rows=np.arange(0,im_row_size,row_size)
    cols=np.arange(0,im_col_size,col_size)[:-1]
    dt = image.dtype
  
    small = np.zeros([len(rows),len(cols)],dtype=dt)

    for i in range(len(rows)):
        for j in range(len(cols)):
            thumb=image[rows[i]:rows[i]+row_size,cols[j]:cols[j]+col_size]
            small[i,j] = np.nanmean(thumb)

    return small

In [ ]:
# downlook road array
roadarray_dl = boxcar_nan(roadarray,(alks,rlks))

In [ ]:
# creates masked unwrapped ifgs
slc1 = np.ndarray([daz,drg],'complex')
slc2 = np.ndarray([daz,drg],'complex')

for k in np.arange(nd-1): 
    # create downlooked, filtered, unwrapped ifgs
    ds1 = gdal.Open(f'{datadir}/SLC_vv/{dates[k]}/{dates[k]}.slc.full', gdal.GA_ReadOnly)
    ds2 = gdal.Open(f'{datadir}/SLC_vv/{dates[k+1]}/{dates[k+1]}.slc.full', gdal.GA_ReadOnly)
    slc1[:,:] = ds1.GetRasterBand(1).ReadAsArray(rg0,az0,drg,daz)
    slc2[:,:] = ds2.GetRasterBand(1).ReadAsArray(rg0,az0,drg,daz)
    # mask to only include road pixels
    slc1_road_mask = slc1*roadarray
    slc2_road_mask = slc2*roadarray
    ifg_road_mask = slc1_road_mask*np.conj(slc2_road_mask)
    # downlook with our function to account for nans
    ifg_road_mask_dl = boxcar_nan(ifg_road_mask, (alks,rlks))
    # read in unw ifg file
    ds = gdal.Open(f'{datadir}/unw_ifgs/{dates[k]}_{dates[k+1]}/{dates[k]}_{dates[k+1]}.unw', gdal.GA_ReadOnly)
    unw_ifg = ds.GetRasterBand(2).ReadAsArray()
    # get mod 2pi ifg
    mod_ifg = np.mod(unw_ifg-np.pi,2*np.pi)-np.pi
    # get unwrappped masked ifg
    unw_road_mask = np.angle(np.exp(1j*mod_ifg)*np.conj(ifg_road_mask_dl))+unw_ifg
    unw_road_mask = np.nan_to_num(unw_road_mask)
    # write to file
    print('write to file')
    if not os.path.isdir(f'{datadir}/realtimeseries/unw_ifgs/masked/{dates[k]}_{dates[k+1]}'):
        os.mkdir(f'{datadir}/realtimeseries/unw_ifgs/masked/{dates[k]}_{dates[k+1]}')
    colds = driver.Create(f'{datadir}/realtimeseries/unw_ifgs/masked/{dates[k]}_{dates[k+1]}/{dates[k]}_{dates[k+1]}_road_mask.unw',np.shape(unw_road_mask)[1],np.shape(unw_road_mask)[0],1,gdal.GDT_Float32)
    colds.GetRasterBand(1).WriteArray(unw_road_mask)
    colds=None
    # create vrt file
    print('create vrt')
    cmd = f'fixImageXml.py -i {datadir}/realtimeseries/unw_ifgs/masked/{dates[k]}_{dates[k+1]}/{dates[k]}_{dates[k+1]}_road_mask.unw -f'
    os.system(cmd)
    print(f'{k} done')